In [ ]:
import boto3


AWS_REGION = 'ap-south-1'

# Create an IAM client to interact with IAM
iam_client = boto3.client('iam', region_name=AWS_REGION)
role_name = 'BrainyPi_sagemaker_tutorial'

policy = {
    'Statement': [
        {
            'Action': 'sts:AssumeRole',
            'Effect': 'Allow',
            'Principal': {'Service': 'sagemaker.amazonaws.com'},
        }],
     'Version': '2012-10-17'
}


In [ ]:
import json

new_role = iam_client.create_role(
    AssumeRolePolicyDocument=json.dumps(policy),
    Path='/',
    RoleName=role_name
)

role_arn = new_role['Role']['Arn']

iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
);

print("Done! Role created. Role ARN: %s" % role_arn)

In [ ]:
# Create an S3 client
s3_client = boto3.client('s3', region_name=AWS_REGION)

# Name buckets
bucket='sagemakertutorial'

# Check if bucket exists
if boto3.resource('s3').Bucket(bucket) not in boto3.resource('s3').buckets.all():
    s3_client.create_bucket(
        Bucket=bucket,
        CreateBucketConfiguration={
            'LocationConstraint': AWS_REGION
        }
    )
else:
    print(f'Bucket {bucket} already exists. No action needed.')

In [ ]:
model_zip_filename = './coco_ssd_mobilenet_v1_1.0.zip'
!curl http://storage.googleapis.com/download.tensorflow.org/models/tflite/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip \
    --output {model_zip_filename}

# Extract model from zip file
!unzip -u {model_zip_filename}

model_filename = 'detect.tflite'
model_name = model_filename.split('.')[0]

# Compress model into .tar.gz so SageMaker Neo can use it
model_tar = model_name + '.tar.gz'
!tar -czf {model_tar} {model_filename}

In [ ]:
# Upload model        
s3_client.upload_file(Filename=model_tar, Bucket=bucket, Key=model_tar)

In [ ]:
s3_input_location = f's3://{bucket}/{model_tar}'
s3_output_location = f's3://{bucket}/output'

framework = 'tflite'
data_shape = '{"normalized_input_image_tensor":[1, 300, 300, 3]}'
target_device = 'rk3399'
# Create a SageMaker client so you can submit a compilation job
sagemaker_client = boto3.client('sagemaker', region_name=AWS_REGION)

# Give your compilation job a name
compilation_job_name = 'brainypiSagemakerTutorial4'
print(f'Compilation job for {compilation_job_name} started')

response = sagemaker_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role_arn,
    InputConfig={
        'S3Uri': s3_input_location,
        'DataInputConfig': data_shape,
        'Framework': framework.upper()
    },
    OutputConfig={
        'S3OutputLocation': s3_output_location,
        'TargetPlatform': {
            'Arch': "ARM64",
            'Os': "LINUX"
        } 
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900
    }
)

# Optional - Poll every 30 sec to check completion status
import time

while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

print(response)

In [ ]:
!pip3 install numpy pillow matplotlib 

!pip install https://neo-ai-dlr-release.s3-us-west-2.amazonaws.com/v1.10.0/rk3399/dlr-1.10.0-py3-none-any.whl

In [ ]:
# Download compiled model locally to edge device
object_path = f'output/{model_name}-LINUX_ARM64.tar.gz'
neo_compiled_model = f'compiled-{model_name}.tar.gz'
s3_client.download_file(bucket, object_path, neo_compiled_model)

# Extract model from .tar.gz so DLR can use it
!mkdir ./dlr_model # make a directory to store your model (optional)
!tar -xzvf ./compiled-detect.tar.gz --directory ./dlr_model

In [ ]:
# Download an image for model to make a prediction
input_image_filename = './input_image.jpg'
!curl https://farm9.staticflickr.com/8463/8132484846_8ce4da18ba_z.jpg --output {input_image_filename}

In [ ]:
import dlr
import numpy as np
import os
import cv2
from dlr.counter.phone_home import PhoneHome

device = 'cpu'
model = dlr.DLRModel('./dlr_model', device)
PhoneHome.disable_feature()

classLabels = []
filename = './labelmap.txt'
with open(filename, 'rt') as spt:
    classLabels = spt.read().rstrip('\n').split('\n')

inputImage = cv2.imread("./input_image.jpg")

# Format image so model can make predictions
#resized_image = image.resize((300, 300))
resized_image = cv2.resize(inputImage, (300, 300), interpolation = cv2.INTER_AREA)

# Model is quantized, so convert the image to uint8
x = np.array(resized_image).astype('uint8')
out = model.run(x)

detection_boxes = np.squeeze(out[0])
detection_classes = np.squeeze(out[1])
detection_scores = np.squeeze(out[2]) 
num_detections = np.squeeze(out[3])

# Loop over the detections
for i in range(num_detections):
    confidence = detection_scores[i]
    if confidence > 0.6:  # You can adjust the confidence threshold as needed
        class_id = int(detection_classes[i])
        class_name = classLabels[class_id + 1]
        box = detection_boxes[i] * np.array([inputImage.shape[1], inputImage.shape[0], inputImage.shape[1], inputImage.shape[0]])
        (startX, startY, endX, endY) = box.astype("int")
        cv2.rectangle(inputImage, (startX, startY), (endX, endY), (255, 0, 0), 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        text = "{}: {:.2f}%".format(class_name, (confidence * 100))
        cv2.putText(inputImage, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display the image with bounding boxes
cv2.imshow("Image with Bounding Boxes", inputImage)
cv2.waitKey(0)
cv2.destroyAllWindows()